# 🤖 DeepSeek R1 Chat Interface


## 📱 [GitHub Repository](https://github.com/edoardoavenia/deepseek-r1-gradio-env)

### Interactive environment to explore DeepSeek-R1 on Google Colab. This project integrates Gradio UI and LangChain pipelines, using Ollama as the model runtime to create a smooth and accessible development experience.

In [ ]:
# Update and install necessary packages

!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

## Model Setup

### This implementation uses DeepSeek-R1 14B (deepseek-r1:14b), the largest model in the DeepSeek-R1 family that can run on Google Colab with T4 GPU (16GB). The model is a Qwen distillation optimized for reasoning.

#### Important: Remember to change the runtime type to T4 GPU on Google Colab before starting.

In [ ]:
import threading
import subprocess
import time

# Start Ollama service in background

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(3)

# Pull DeepSeek model and install required packages

!ollama pull deepseek-r1:14b

!pip install langchain_core langchain_ollama gradio

## 💬 Conversational Interface

### The following code implements a chat interface that showcases DeepSeek-R1's unique capabilities:
- Separates the model's reasoning from its final answer using XML parsing
- Maintains conversation history with LangChain
- Provides a clean, user-friendly interface through Gradio
- Uses a temperature of 0.4

#### Model info and other variants available on [Ollama Library ↗](https://ollama.com/library/deepseek-r1:14b)


In [ ]:
import gradio as gr
from langchain_ollama import ChatOllama
import xml.etree.ElementTree as ET
from langchain_core.messages import HumanMessage, AIMessage

# Initialize model
llm = ChatOllama(model="deepseek-r1:14b", temperature=0.4)

# Conversation memory
messages = []

def respond(message, history):
    # Add user message to history
    messages.append(HumanMessage(content=message))

    # Get model response
    response = llm.invoke(messages)

    # Parse response
    reasoning, answer = parse_message(response.content)

    # Add AI response to history
    messages.append(AIMessage(content=answer))

    return f"# Reasoning:\n{reasoning}\n---\n# Answer:\n{answer}"

def parse_message(content):
    try:
        # Add root tag for valid XML
        xml_content = f"<root>{content}</root>"
        root = ET.fromstring(xml_content)

        # Extract think content
        think_content = root.find('think').text if root.find('think') is not None else ""

        # Extract answer content
        answer_content = "".join(root.itertext())
        answer_content = answer_content.replace(think_content, "", 1).strip()

        return think_content.strip(), answer_content

    except ET.ParseError:
        return "", content
    except Exception:
        return "", content

# Initialize Gradio interface
demo = gr.ChatInterface(
    fn=respond,
    title="DeepSeek R1 Chat",
    description="DeepSeek R1 • Ollama • LangChain • Gradio",
    theme="default"
)

# Launch the interface
demo.launch()